<a href="https://colab.research.google.com/github/altaga/Open-Driving-Monitor/blob/main/Drowsiness/train/Train_Test_and_Deploy_Blink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing modules

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
from keras.layers import Conv2D, Flatten, Dense, AveragePooling2D, Dropout, MaxPooling2D,Activation,BatchNormalization
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.regularizers import l2
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from sklearn.model_selection import train_test_split
import glob

Download Dataset and Haarcascades

In [ ]:
!wget https://github.com/altaga/DBSE-monitor/raw/master/Drowsiness/train/dataset/dataset_B_Eye_Images.zip -O temp.zip
!unzip temp.zip
!rm temp.zip
!wget https://github.com/altaga/DBSE-monitor/blob/master/Drowsiness/train/haar_models/haarcascade_eye.xml
!wget https://github.com/altaga/DBSE-monitor/blob/master/Drowsiness/train/haar_models/haarcascade_frontalface_default.xml

Setting up training

In [ ]:
NUM_CLASSES = 2
IMG_SIZE = 24
# Training Parameters
n_epochs = 20
batch_size = 64
weight_decay = 1e-4

Setup support functions for processing and training.

In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    print("-" * 50)
    print("Frozen model layers: ")
    layers = [op.name for op in import_graph.get_operations()]
    if print_graph == True:
        for layer in layers:
            print(layer)
    print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

Creating the model and training with the entire data set.

In [ ]:
images = []
labels = []
files = list(map(lambda x: {'file': x, 'label':1}, glob.glob('dataset_B_Eye_Images/openRightEyes/*.jpg')))
files.extend(list(map(lambda x: {'file': x, 'label':1}, glob.glob('dataset_B_Eye_Images/openLeftEyes/*.jpg'))))
files.extend(list(map(lambda x: {'file': x, 'label':0}, glob.glob('dataset_B_Eye_Images/closedLeftEyes/*.jpg'))))
files.extend(list(map(lambda x: {'file': x, 'label':0}, glob.glob('dataset_B_Eye_Images/closedRightEyes/*.jpg'))))

for file in files:
    img = cv2.imread(file['file'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    images.append(np.array(img))
    labels.append(file['label'])

labels = np_utils.to_categorical(labels, NUM_CLASSES)
images = np.array(images)
images = images.astype('float32') / 255.0

print(labels.shape)
print(images.shape)

# Data Split
X_train, X_test, y_train, y_test = train_test_split(images,labels,
                                  random_state=104,
                                  test_size=0.4,
                                  shuffle=True)

X_valid, X_test, y_valid, y_test = train_test_split(X_test,y_test ,
                              random_state=104,
                              test_size=0.3,
                              shuffle=True)

# Model
model = Sequential()
# 1st convolution layer
model.add(Conv2D(64, (8,8), activation='relu', input_shape=(24, 24, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
# 1st Dropout
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.1))
# 2nd convolution layer
model.add(Conv2D(128, (8,8), padding='same', kernel_regularizer= l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
# 2nd Dropout
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))
# Flatten
model.add(Flatten())
# 1st Dense
model.add(Dense(512, activation="linear"))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation="linear"))
model.add(Activation('elu'))
model.add(Dropout(0.2))
# Output Layer
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train,
                    validation_data=(X_valid, y_valid),
                    epochs=n_epochs, batch_size=batch_size)

score = model.evaluate(X_test, y_test, batch_size=batch_size)

print("Training Score: {}".format(score))
model.fit(X_test, y_test,
                    validation_data=(X_valid, y_valid),
                    epochs=int(n_epochs/2), batch_size=batch_size)
model.fit(X_valid, y_valid, epochs=int(n_epochs/2), batch_size=batch_size)

print("Model trained with all data")

Download test image

In [ ]:
!wget https://raw.githubusercontent.com/altaga/Open-Driving-Monitor/main/Drowsiness/test/testImages/open1.png

Testing the model

In [ ]:
# load the image from disk
class_names = [
    'Close',
    'Open',
]

image = cv2.imread('open.png')
imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
imageGray3 = cv2.cvtColor(imageGray, cv2.COLOR_GRAY2BGR)
imageGray3res = cv2.resize(imageGray3, (24, 24),  interpolation=cv2.INTER_AREA)
imageNP = np.array(imageGray3res)
imageNormal = imageNP.astype('float32') / 255.0
plt.imshow(imageRGB)
data = np.expand_dims(imageNormal, 0)
outputs = model.predict(data)
# Get Values
final_outputs = outputs[0]
# get the class label
label_id = np.argmax(final_outputs)
out_name = class_names[label_id]
print(out_name)

Creating a forzen graph that can be used in OpenCV DNN

In [ ]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name="emotion"))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)